<a href="https://colab.research.google.com/github/Kieil/Used-cars/blob/main/Used_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from xgboost import XGBRegressor
import xgboost as xgb
model1 = XGBRegressor()
import lightgbm as lgb

#zrobic luksusowe marki binarnie
#zabytki
#interior grouping
#wyciagnac paliwo z informacji silnika

In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

test= pd.read_csv('test.csv')
train= pd.read_csv('train.csv')

train = train[~train['id'].isin(['113115', '50492'])]
all_data = pd.concat([train, test])
test.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [ ]:
def extract_data_from_engine(train):
    train=train.copy()
    train['horsepower'] = train['engine'].str.extract(r'(\d+\.\d+)(?=HP)').astype(float)
    train['engine_size'] = train['engine'].str.extract(r'(\d+\.\d+)(?=L)').astype(float)
    train['cylinders'] = train['engine'].str.extract(r'(\d+)\s(Cylinder|V\d|Straight)')[0].astype(float)
 #filling empty in fuel column
   # train['engine_extract'] = train['engine'].str.extract(r'Engine\s*(.*)')[0]
   # train['engine_extract'] =
    return train
train = extract_data_from_engine(train)
test = extract_data_from_engine(test)

In [ ]:
#train['engine_extract'].unique()

In [ ]:
# Fill missing values in 'fuel_type' with corresponding values from 'engine_extract'
#train['fuel_type'] = train['fuel_type'].fillna(train['engine_extract'])
#test['fuel_type'] = test['fuel_type'].fillna(test['engine_extract'])

In [ ]:
current_year = 2024
train['Vehicle_Age'] = current_year - train['model_year']
test['Vehicle_Age'] = current_year - test['model_year']
train['Mileage_per_Year'] = train['milage'] / (train['Vehicle_Age'] + 10e-5)
test['Mileage_per_Year'] = test['milage'] / (test['Vehicle_Age'] + 10e-5)

train['brand'] = train['brand'].str.lower()
test['brand'] = test['brand'].str.lower()

train['transmission'] = train['transmission'].str.lower()
test['transmission'] = test['transmission'].str.lower()

train['int_col'] = train['int_col'].str.lower()
test['int_col'] = test['int_col'].str.lower()

In [ ]:
train['antique'] = train['Vehicle_Age'].apply(lambda x: 1 if x < 20 else 0)
test['antique'] = test['Vehicle_Age'].apply(lambda x: 1 if x < 20 else 0)

In [ ]:
luxury_brands = ['lamborghini', 'porsche', 'bugatti', 'rolls-royce', 'bentley', 'mclaren', 'ferrari', 'aston','rivian']
train['luxury'] = train['brand'].apply(lambda x: 1 if x in luxury_brands else 0)
test['luxury'] = test['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

In [ ]:
train['transmission_auto'] = train['transmission'].apply(lambda x: 1 if 'automatic' in x or 'a/t' in x else 0)
test['transmission_auto'] = test['transmission'].apply(lambda x: 1 if 'automatic' in x or 'a/t' in x else 0)

In [ ]:
train['luxury'].unique()

array([0, 1])

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming train1 and test1 are your DataFrames
object_cols = ['brand', 'fuel_type', 'int_col','ext_col', 'accident', 'clean_title','transmission','accident']

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Apply label encoding to each object column in both DataFrames
for col in object_cols:
    train[col] = label_encoder.fit_transform(train[col])
    test[col] = label_encoder.transform(test[col])

In [ ]:
train1 = train.drop(['id', 'model','engine','price'],axis =1)
test1 = test.drop(['id','model','engine'], axis = 1)

In [ ]:
test1.head()

,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,horsepower,engine_size,cylinders,Vehicle_Age,Mileage_per_Year,antique,luxury,transmission_auto
0,26,2015,98000,2,16,304,8,1,0,240.0,2.0,4.0,9,10888.767903,1,0,1
1,26,2020,9142,3,29,263,12,1,0,395.0,3.0,6.0,4,2285.442864,1,0,1
2,14,2022,28121,2,3,304,55,1,1,NaN,3.5,NaN,2,14059.797010,1,0,1
3,3,2016,61258,2,38,261,12,1,1,NaN,NaN,NaN,8,7657.154286,1,0,1
4,3,2018,59000,2,37,128,12,1,0,252.0,2.0,4.0,6,9833.169447,1,0,1


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [ ]:
best_params = {
    'n_estimators': 1225,
    'num_leaves': 137,
    'max_depth': 14,
    'cat_smooth': 96,
    'learning_rate': 0.0023941644225363256,
    'subsample': 0.9082095260228584,
    'colsample_bytree': 0.6165900236226695,
    'min_split_gain': 0.0308677316309982,
    'min_child_weight': 68,
    'lambda_l2': 1.7319600391087514e-07,
    'lambda_l1': 8.761594422544116e-07,
    'max_bin': 749,
    'objective': 'regression',
    'metric': 'rmse',
    'random_state': 42,
    'boosting_type': 'gbdt',
}

In [ ]:
lgb_model = lgb.LGBMRegressor(**best_params)

callbacks = [
    lgb.early_stopping(stopping_rounds=100),
    lgb.log_evaluation(100)
]

In [ ]:
fit1 = train['price']

In [ ]:
lgb_model.fit(
    train1, fit1,
    eval_set=[(train1, fit1)],
    eval_metric='rmse',
    callbacks=callbacks
)

[LightGBM] [Warning] lambda_l2 is set=1.7319600391087514e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7319600391087514e-07
[LightGBM] [Warning] lambda_l1 is set=8.761594422544116e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.761594422544116e-07
[LightGBM] [Warning] lambda_l2 is set=1.7319600391087514e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7319600391087514e-07
[LightGBM] [Warning] lambda_l1 is set=8.761594422544116e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.761594422544116e-07
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039195 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2579
[LightGBM] [Info] Number of data points in the train set: 188533, number of used features: 17
[LightGBM] [Info] Start training from score 43878.016178


/usr/local/lib/python3.10/dist-packages/lightgbm/callback.py:341: UserWarning: Only training set found, disabling early stopping.
  _log_warning("Only training set found, disabling early stopping.")


[100]	training's rmse: 76512.5
[200]	training's rmse: 74926.9
[300]	training's rmse: 73798.5
[400]	training's rmse: 72984.4
[500]	training's rmse: 72375.3
[600]	training's rmse: 71915.3
[700]	training's rmse: 71546.4
[800]	training's rmse: 71240.9
[900]	training's rmse: 70982.2
[1000]	training's rmse: 70761.3
[1100]	training's rmse: 70559
[1200]	training's rmse: 70376.3


LGBMRegressor(cat_smooth=96, colsample_bytree=0.6165900236226695,
              lambda_l1=8.761594422544116e-07, lambda_l2=1.7319600391087514e-07,
              learning_rate=0.0023941644225363256, max_bin=749, max_depth=14,
              metric='rmse', min_child_weight=68,
              min_split_gain=0.0308677316309982, n_estimators=1225,
              num_leaves=137, objective='regression', random_state=42,
              subsample=0.9082095260228584)

In [ ]:
#fit1 = train['price']

# Fit the model
#model1 = xgb.train
#model1.fit(train1,fit1)


In [ ]:
pred1=lgb_model.predict(test1)

[LightGBM] [Warning] lambda_l2 is set=1.7319600391087514e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7319600391087514e-07
[LightGBM] [Warning] lambda_l1 is set=8.761594422544116e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.761594422544116e-07


In [ ]:
submission = pd.DataFrame({
    'id': test['id'],
    'price': pred1.astype(int)  # Convert predictions to integers
})

In [ ]:
submission.to_csv('submission_car.csv', index=False)

In [ ]:
submission

,id,price
0,188533,18930
1,188534,74888
2,188535,53007
3,188536,32588
4,188537,30800
...,...,...
125685,314218,27607
125686,314219,44857
125687,314220,21838
125688,314221,17430
